# Project Proposal
Before you start writing any code, your group should outline the scope and purpose of your project. This helps provide direction and prevent scope creep.
Write this as a brief summary of your interests and intent, including:


The kind of data you'd like to work with/field you're interested in (e.g., geodata, weather data, etc.)


The kinds of questions you'll be asking of that data


Possible source for such data

**READ ME:  You must run this in your Terminal** To see the map: jupyter nbextension enable --py gmaps

- Cool Map 
- import gmaps
- import gmaps.datasets
- from geopy.geocoders import Nominatim
- geolocator = Nominatim(user_agent="meliswright925")
- Configure gmaps
- gmaps.configure(api_key=gkey)

In [1]:
%matplotlib inline


In [2]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
# Google API Key
from config import gkey

#Cool Map 
import gmaps
import gmaps.datasets
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="meliswright925")
# Configure gmaps
gmaps.configure(api_key=gkey)

# Google developer API key
from config import gkey



In [3]:
# Create a list of times to find commute duration for
times_list_to_work = [5,6,7,8,9,10,11,12]
times_list_to_home = [13,14,15,16,17,18,19]
dates_list = ['3/2/2020','3/3/2020','3/4/2020','3/5/2020','3/6/2020','3/7/2020','3/8/2020']
home_location = 'Moffett Field, CA'
work_location = 'Salesforce Tower'

#If we wanted to change the home and work locations, we'd simply add an input form to 
#home_location = input ("Where is your Home (Example: Moffett Field, CA)")
#work_location = input ("Where is your work (Example: Salesforce Tower, CA)")


In [4]:
# create a new list with combined dates/times into epoch time to be used to query database
pattern = '%m/%d/%Y-%H'
date_string = ''
datetime_list_to_work = []
datetime_list_to_home = []
times_df = pd.DataFrame(columns=["Commute Date", "Commute Start", "Epoch Time"])

for date_in in dates_list:
    
    for time_in in times_list_to_work:
        date_string = date_in + '-' + str(time_in)
        epoch = int(time.mktime(time.strptime(date_string, pattern)))
        datetime_list_to_work.append(epoch)
        times_df = times_df.append({
             "Commute Date": date_in,
             "Commute Start": time_in,
             "Epoch Time": epoch
          }, ignore_index=True)
        
    for time_in in times_list_to_home:
        date_string = date_in + '-' + str(time_in)
        epoch = int(time.mktime(time.strptime(date_string, pattern)))
        datetime_list_to_home.append(epoch)
        times_df = times_df.append({
             "Commute Date": date_in,
             "Commute Start": time_in,
             "Epoch Time": epoch
          }, ignore_index=True)

In [5]:
#Script to get the map to show
#sources: https://stackoverflow.com/questions/13686001/python-module-for-getting-latitude-and-longitude-from-the-name-of-a-us-city
#sources: https://geopy.readthedocs.io/en/stable/
#**You must run this in your Terminal**  jupyter nbextension enable --py gmaps

home = geolocator.geocode(home_location)
#print(home.raw)
work = geolocator.geocode(work_location)
#print(work.raw)

#Get the Lattitudes and Longitudes for the map
home_coord = (home.latitude, home.longitude)

work_coord = (work.latitude, work.longitude)

#Display the Map

fig = gmaps.figure()
directions = gmaps.directions_layer(home_coord, work_coord)
fig.add_layer(directions)
fig


Figure(layout=FigureLayout(height='420px'))

In [6]:
print(datetime_list_to_work)
print(datetime_list_to_home)
times_df 

[1583154000, 1583157600, 1583161200, 1583164800, 1583168400, 1583172000, 1583175600, 1583179200, 1583240400, 1583244000, 1583247600, 1583251200, 1583254800, 1583258400, 1583262000, 1583265600, 1583326800, 1583330400, 1583334000, 1583337600, 1583341200, 1583344800, 1583348400, 1583352000, 1583413200, 1583416800, 1583420400, 1583424000, 1583427600, 1583431200, 1583434800, 1583438400, 1583499600, 1583503200, 1583506800, 1583510400, 1583514000, 1583517600, 1583521200, 1583524800, 1583586000, 1583589600, 1583593200, 1583596800, 1583600400, 1583604000, 1583607600, 1583611200, 1583668800, 1583672400, 1583676000, 1583679600, 1583683200, 1583686800, 1583690400, 1583694000]
[1583182800, 1583186400, 1583190000, 1583193600, 1583197200, 1583200800, 1583204400, 1583269200, 1583272800, 1583276400, 1583280000, 1583283600, 1583287200, 1583290800, 1583355600, 1583359200, 1583362800, 1583366400, 1583370000, 1583373600, 1583377200, 1583442000, 1583445600, 1583449200, 1583452800, 1583456400, 1583460000, 15

,Commute Date,Commute Start,Epoch Time
0,3/2/2020,5,1583154000
1,3/2/2020,6,1583157600
2,3/2/2020,7,1583161200
3,3/2/2020,8,1583164800
4,3/2/2020,9,1583168400
...,...,...,...
100,3/8/2020,15,1583704800
101,3/8/2020,16,1583708400
102,3/8/2020,17,1583712000
103,3/8/2020,18,1583715600


In [7]:
def get_commute_times(start_point, end_point, datetime_list):
    # Base info for google maps direction request
    # create a params dict that will be updated for each date/time
    params = {"key": gkey}
    base_url = "https://maps.googleapis.com/maps/api/directions/json"
    params['mode'] = 'driving'
    params['origin'] = start_point
    params['destination'] = end_point

    # Create dictionary to store {date_time: commute_time}
    commute_dict = {}
    
    # Loop through the date/times to get the period you are going to search for
    for commute_start in datetime_list:

        # update address key value
        params['departure_time'] = commute_start

        # make request
        commute_info = requests.get(base_url, params=params)
    
        # convert to json
        commute_info = commute_info.json()
    
        commute_time_legs = commute_info['routes'][0]['legs']
        total_route_time = 0
        for c_time in commute_time_legs:
            total_route_time = total_route_time + c_time['duration_in_traffic']['value']

        commute_dict[commute_start] = total_route_time
        time_here = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(commute_start))
        # print(f"Commute duration from {start_point} to {end_point} for {time_here} = {total_route_time/60} minutes")

        
    return commute_dict



In [8]:
# Get commute times for going to work
commute_to_work_dict = get_commute_times(home_location, work_location, datetime_list_to_work)

In [9]:
#commute_to_work_dict

In [10]:
# Create dataframe to store commute data to work
commute_to_work_df = pd.DataFrame.from_dict(commute_to_work_dict, orient='index', columns=['Commute Time'])
commute_to_work_df.index.name = 'Epoch Time'
commute_to_work_df.head()

,Commute Time
Epoch Time,
1583154000,2503
1583157600,2768
1583161200,3870
1583164800,4416
1583168400,3781


In [11]:
# Get commute times for going home
commute_to_home_dict = get_commute_times(work_location, home_location, datetime_list_to_home)

KeyboardInterrupt: 

In [ ]:
#commute_to_home_dict 

In [ ]:
# Create dataframe to store commute data to home
commute_to_home_df = pd.DataFrame.from_dict(commute_to_home_dict, orient='index', columns=['Commute Time'])
commute_to_home_df.index.name = 'Epoch Time'
commute_to_home_df.head()

In [ ]:
commute_to_work_df = pd.merge(commute_to_work_df,
                 times_df[['Commute Date', 'Commute Start', 'Epoch Time']],
                 on='Epoch Time')
commute_to_work_df.head()


In [ ]:
commute_to_home_df = pd.merge(commute_to_home_df,
                 times_df[['Commute Date', 'Commute Start', 'Epoch Time']],
                 on='Epoch Time')
commute_to_home_df.head()


In [ ]:
# Add Commute To/From info as a column
commute_to_work_df['Commute Name'] = f"{home_location } to {work_location}"
commute_to_work_df.head()

In [ ]:
# Add Commute To/From info as a column
commute_to_work_df['Commute Name'] = f"{home_location} to {work_location}"
commute_to_work_df.head()

In [ ]:
commute_to_home_df['Commute Name'] = f"{work_location} to {home_location}"
commute_to_home_df.head()



In [ ]:
#Convert the seconds to minutes on commute 
commute_to_work_df['Minutes'] = commute_to_work_df['Commute Time']/60
days_of_week = ['Monday','Tuesday', 'Wednesday','Thursday','Friday','Saturday','Sunday']
# Split up our data into groups based upon 'date'
work_group = commute_to_work_df.groupby('Commute Date')
# Find out how long each day took 
work_trips = work_group['Minutes'].mean()
# Drop the 'stoptime' row that is contained within our group
#gender_trips = gender_trips.drop(gender_trips.index[3])
# Chart our data, give it a title, and label the axes
work_chart = work_trips.plot(kind="bar", title="Commute Duration by Date (AM)", color ='orange', rot=45)
work_chart.set_xlabel("Dates")
work_chart.set_ylabel("Duration (in minutes)")
work_chart.grid(b=True)
work_chart.set_xticklabels(days_of_week)

# Create a bar chart based upon the above data
plt.show()

In [ ]:
#Convert the seconds to minutes on commute 
commute_to_home_df['Minutes'] = commute_to_home_df['Commute Time']/60

#Split up our data into groups based upon 'date'
home_group = commute_to_home_df.groupby('Commute Date')

# Find out how long each day took 
home_trips = home_group['Minutes'].mean()

# Drop the 'stoptime' row that is contained within our group
#gender_trips = gender_trips.drop(gender_trips.index[3])

# Chart our data, give it a title, and label the axes
home_chart = home_trips.plot(kind="bar", title="Commute Duration by Date (PM)", rot = 45)
home_chart.set_xlabel("Dates")
home_chart.set_ylabel("Minutes")
home_chart.grid(b=True)
home_chart.set_xticklabels(days_of_week)

# Create a bar chart based upon the above data
plt.show()

In [ ]:
#Convert the seconds to minutes on commute 
commute_to_home_df['Minutes'] = commute_to_home_df['Commute Time']/60

#Split up our data into groups based upon 'date'
home_group = commute_to_home_df.groupby('Commute Date')

# Find out how long each day took 
home_trips = home_group['Minutes'].mean()

home_trips.values

In [ ]:
#print (gn.geocode("Sacramento"))

In [ ]:


# Drop the 'stoptime' row that is contained within our group
#gender_trips = gender_trips.drop(gender_trips.index[3])

# Chart our data, give it a title, and label the axes
#home_chart = home_trips.plot(kind="line", title="Commute Time Home", rot = 35)
#home_chart.set_xlabel("Day of the Week")
#home_chart.set_ylabel("Amount of Time Spent in Commute")

#multiple
#plt.plot(home_trips.index, home_trips.values,  marker='o', markerfacecolor='blue', markersize=12, color='red', linewidth=4)
#plt.plot(home_trips.index, home_trips2.values,  marker='o', markerfacecolor='blue', markersize=12, color='skyblue', linewidth=4)
#plt.plot(home_trips3.index, home_trips3.values,  marker='^', markerfacecolor='blue', markersize=12, color='red', linewidth=4)

# Create a bar chart based upon the above data
#plt.show()

In [ ]:
# Save Data to csv
commute_to_work_df.to_csv('commute_to_work.csv')
commute_to_home_df.to_csv('commute_to_home.csv')
# Visualize to confirm commute time data appears
#There is a way to save and update the .csv file but if we don't hard code it in,it will override the data 
#Run a for loop to creae a new csv file and merge them to combine the data! 


In [ ]:
# Going to work...
# Create 7 bar charts, one for each day of week, with start times we have stored as x-axis 
# and commute time in minutes as y-axis

In [ ]:
# Going Home...
# Create 7 bar charts, one for each day of week, with start times we have stored as x-axis 
# and commute time in minutes as y-axis